# AttRec

Next Item Recommendation with Self-Attention (Shuai Zhang et al.) 
[https://arxiv.org/pdf/1808.06414](https://arxiv.org/pdf/1808.06414)

Implement in TensorFlow

## The AttRec algorithm


The proposed self-attentive sequential recommendation model, named **AttRec**.


Suppose the user's short-term intents can be derived from her recent $L$ (e.g., 5, 10) interactions. Assuming each item can be represented with a $d$-dimension embedding vector. Let $X \in \mathbb{R}^{N \times d}$ denote the embedding representations for the whole item set. The latest $L$ items (i.e., from item $t - L + 1$ to item $t$) are stacked together in sequence to get the following matrix:


\begin{equation}
X_t^u = 
\begin{bmatrix}
X_{(t-L+1)1} & X_{(t-L+1)2} & \cdots & X_{(t-L+1)d} \\
\vdots & \vdots & \ddots & \vdots \\
X_{(t-1)1} & X_{(t-1)2} & \cdots & X_{(t-1)d} \\
X_{t1} & X_{t2} & \cdots & X_{td}
\end{bmatrix}
\tag{1}
\end{equation}


Here, the latest $L$ items is a subset of $H^u$. **Query**, **key**, and **value** for user $u$ at time step $t$ in the self-attention model are equal to $X_t^u$.

First, we project **query** and **key** to the same space through a non-linear transformation with shared parameters:

$$
Q' = \text{ReLU}(X_t^u W_Q) \tag{2}
$$

$$
K' = \text{ReLU}(X_t^u W_K) \tag{3}
$$

The weight matrices $W_Q \in \mathbb{R}^{d \times d}$ and $W_K \in \mathbb{R}^{d \times d}$ are used for projecting the **query** and **key**, respectively. The activation function ReLU is applied to introduce non-linearity into the attention mechanism. The affinity matrix is computed as follows:



$$
s_t^u = \text{softmax}\left(\frac{Q'K'^T}{\sqrt{d}}\right) \tag{4}
$$

The output is an $L \times L$ affinity matrix (or attention map) representing the similarity between the $L$ items. The term $\sqrt{d}$ is included to scale the dot-product attention, preventing it from growing excessively large when $d$ is set to a high value (e.g., 100). This scaling helps to reduce issues caused by extremely small gradients. Additionally, a masking operation is applied to the diagonal of the affinity matrix before the softmax step, ensuring that identical vectors for **query** and **key** do not result in high matching scores.

Next, the **value** is kept unchanged, equal to $X_t^u$. Unlike other approaches, where the **value** is often transformed linearly, this model benefits from using identity mapping for **value**. In various applications, such as natural language processing or image recognition, the **value** typically consists of pre-trained embeddings (e.g., word or image features). In this model, the **value** directly reflects parameters specific to the task.

Adding linear (or nonlinear) transformation will increase the difficulty in seeing the actual parameters. Query and key are used as auxiliary factors so that they are not as sensitive as value to transformations. Finally, the affinity matrix and the value are multiplied to form the final weighted output of the self-attention module. 

$$a_{t}^{u}=s_{t}^{u}X_{t}^{u}  \tag{5}$$

The attentive output $a_{t}^{u}\in\mathcal{R}^{L\times d}$ can be interpreted as user's short-term intent representations. To learn a single attentive representation, we average the L self-attention representations, representing user temporal intent. Other aggregation operations like sum, max, and min can also be used, and we will evaluate their effectiveness in our experiments. 

$$m_{t}^{u}=\frac{1}{L}\sum_{l=1}^{L}a_{tl}^{u}  \tag{6}$$

The above attention model lacks time signals. Without them, the input becomes a bag of embeddings and fails to capture sequential patterns. Inspired by the Transformer, we propose to include time information in the query and key using positional embeddings. 

We employ a geometric sequence of timescales to add sinusoids of different frequencies to the input. The time embedding (TE) consists of two sinusoidal signals defined as follows:

$$TE(t,2i)=sin(t/10000^{2i/d})  \tag{7}$$


$$TE(t,2i+1)=cos(t/10000^{2i/d}) \tag{8}$$

Here, t represents the time step and i is the dimension. The TE is simply added to query and key before the nonlinear transformation.

## 3.3 User Long-Term Preference Modelling

To better understand and predict user behavior, we need to consider not only their immediate preferences but also their long-term tastes. Similar to how latent factor models work, we can assign each user and item a set of hidden features or representations. These representations, denoted as $U$ and $V$ respectively, capture the underlying characteristics of users and items.

We could use the dot product to model how much a user likes an item, similar to how latent factor models work. However, recent research suggests that this approach can lead to suboptimal results because it doesn't always follow the expected mathematical properties of distance measures. To address this, we use Euclidean distance to measure how similar a user and an item are.

$$
||U_u - V_i||_2^2 \tag{9}
$$

The distance is expected to be small if user $u$ liked the item $i$, and large otherwise.

## 3.4 Model Learning

**Objective Function:** Given the user's current interests and their long-term preferences, our goal is to predict the next item they will interact with. To ensure consistency, we use Euclidean distance to measure both short-term and long-term similarity, and then combine them to create a final recommendation score. 

$$y_{t+1}^{u}=\omega||U_{u}-V_{\mathcal{H}_{t+1}}^{u}||_{2}^{2}+(1-\omega)||m_{t}^{u}-X_{t+1}^{u}||_{2}^{2} \tag{10}$$

We can extend our model to predict multiple items instead of just one. This allows us to capture sequential patterns and user behavior that involves skipping items. We denote the next T items that the user liked as $T^+$. To train our model, we use a pairwise ranking approach. This involves sampling T negative items that the user did not interact with (or disliked) and denoting this set as $T^-$. 

We use a margin-based hinge loss to encourage discrimination between positive and negative user-item pairs:

$$
L(\Theta) = \sum_{(u,i) \in T^+} \sum_{(u,j) \in T^-} [y_{i}^u + \gamma - y_{j}^u]_+ + \lambda ||\Theta||_2^2 \tag{11}
$$

Here, $\Theta = \{X, V, U, W_Q, W_K\}$ represents the model parameters. $y$ is the margin that separates positive and negative pairs. We use $l_2$ loss to control model complexity. Dropout can be applied to the nonlinear layer of the self-attention module. 

To handle sparse datasets, we can use norm clipping to constrain the parameters $X$, $V$, and $U$ to a unit Euclidean ball:

$$
||X||_2 \leq 1, ||V||_2 \leq 1, ||U||_2 \leq 1 \tag{12}
$$

This regularization approach helps alleviate the curse of dimensionality and prevents data points from spreading too widely.

## Global settings

#### Module imports



In [7]:
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
tf.get_logger().setLevel('ERROR')  # only show error messages
import sys
import os
sys.path.append('..')
os.environ["CUDA_VISIBLE_DEVICES"]='1'

from recommenders.datasets import movielens
from recommenders.models.attrec.attrec import AttRec
from recommenders.models.attrec.dataIterator import DataIterator
from recommenders.models.attrec.create_train_test import create_train_test
from recommenders.utils.constants import SEED as DEFAULT_SEED

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")
print("AttRec module imported successfully!")


System version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 08:22:19) [Clang 14.0.6 ]
Tensorflow version: 2.18.0
AttRec module imported successfully!


#### Global variables

In [8]:
# top k items to recommend
TOP_K = 50

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 30
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results

#### Load Dataset

In [9]:
df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating", "timestamp"]
)

df.head()

100%|██████████| 4.81k/4.81k [00:02<00:00, 1.90kKB/s]


,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


#### Arguments

In [10]:

import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--file_path', type=str, default='/Users/leeisbadk/recommenders/examples/99_model_attrec/ml-100k/ml-100k/u.data', help='training data dir')
parser.add_argument('--test_path', type=str, default='input/test.csv', help='testing data dir')
parser.add_argument('--train_path', type=str, default='input/train.csv', help='training data dir')
parser.add_argument('--mode', type=str, default='train', help='train or test')
parser.add_argument('--w', type=float, default=0.3, help='The final score is a weighted sum of them with the controlling factor ω')
parser.add_argument('--num_epochs', type=int, default=30, help='number of epochs')
parser.add_argument('--sequence_length', type=int, default=5, help='sequence length')
parser.add_argument('--target_length', type=int, default=3, help='target length')
parser.add_argument('--neg_sample_count', type=int, default=10, help='number of negative sample')
parser.add_argument('--item_count', type=int, default=1685, help='number of items')
parser.add_argument('--user_count', type=int, default=945, help='number of user')
parser.add_argument('--embedding_size', type=int, default=100, help='embedding size')
parser.add_argument('--batch_size', type=int, default=256, help='batch size')
parser.add_argument('--learning_rate', type=float, default=1e-2, help='learning rate')
parser.add_argument('--keep_prob', type=float, default=0.5, help='keep prob of dropout')
parser.add_argument('--l2_lambda', type=float, default=1e-3, help='Regularization rate for l2')
parser.add_argument('--gamma', type=float, default=0.5, help='gamma of the margin higle loss')
parser.add_argument('--grad_clip', type=float, default=10, help='gradient clip to prevent from grdient to large')
parser.add_argument('--save_path', type=str, default='save_path/model1.ckpt', help='the whole path to save the model')

FLAGS, unparsed = parser.parse_known_args()

print(FLAGS)





Namespace(file_path='/Users/leeisbadk/Library/Jupyter/runtime/kernel-v3b6e6cf3dd941b0ab5ee4c1225be15d1ad3028b69.json', test_path='input/test.csv', train_path='input/train.csv', mode='train', w=0.3, num_epochs=30, sequence_length=5, target_length=3, neg_sample_count=10, item_count=1685, user_count=945, embedding_size=100, batch_size=256, learning_rate=0.01, keep_prob=0.5, l2_lambda=0.001, gamma=0.5, grad_clip=10, save_path='save_path/model1.ckpt')


#### Metrics

In [11]:
def Metric_HR(target_list, predict_list, num):
    count = 0
    for i in range(len(target_list)):
        t = target_list[i]
        preds = predict_list[i]
        preds = preds[:num]
        if t in preds:
            count += 1
    return count / len(target_list)

def Metric_MRR(target_list, predict_list):

    count = 0
    for i in range(len(target_list)):
        t = target_list[i]
        preds = predict_list[i]
        rank = preds.index(t) + 1
        count += 1 / rank
    return count / len(target_list)

## Model

In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def main(args):
    data, num_users, num_items = df, df['userID'].nunique(), df['itemID'].nunique()
    print(' make datasets')
    train_data, test_data ,user_all_items, all_user_count\
        , all_item_count, user_map, item_map \
        = create_train_test(data, FLAGS.sequence_length, FLAGS.target_length, is_Save=True)
    FLAGS.item_count = all_item_count
    FLAGS.user_count = all_user_count
    all_index = [i for i in range(FLAGS.item_count)]
    print(train_data)
    print(test_data)
    print(' load model and training')
    graph = tf.Graph()
    with graph.as_default():
      with tf.compat.v1.Session() as sess:
          #Load model
          model = AttRec(FLAGS)
          topk_index = model.predict(all_index,len(all_index))
          total_loss = model.loss

          #Add L2
          # with tf.name_scope('l2loss'):
          #     loss = model.loss
          #     tv = tf.trainable_variables()
          #     regularization_cost = FLAGS.l2_lambda * tf.reduce_sum([tf.nn.l2_loss(v) for v in tv])
          #     total_loss = loss + regularization_cost

          #Optimizer
          global_step = tf.Variable(0, trainable=False)
          update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
          with tf.control_dependencies(update_ops):
              optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)
              tvars = tf.trainable_variables()
              grads, _ = tf.clip_by_global_norm(tf.gradients(total_loss, tvars), FLAGS.grad_clip)
              grads_and_vars = tuple(zip(grads, tvars))
              train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)


          #Saver and initializer
          saver = tf.train.Saver()
          if FLAGS.mode == 'test':
              saver.restore(sess, FLAGS.save_path)
          else:
              sess.run(tf.global_variables_initializer())

          #Batch reader
          trainIterator = DataIterator(data=train_data
                                      , batch_size=FLAGS.batch_size
                                      ,max_seq_length=FLAGS.batch_size
                                      ,neg_count=FLAGS.neg_sample_count
                                      ,all_items=all_index
                                      ,user_all_items=user_all_items
                                      ,shuffle=True)
          testIterator = DataIterator(data=test_data
                                      ,batch_size = FLAGS.batch_size
                                      , max_seq_length=FLAGS.batch_size
                                      , neg_count=FLAGS.neg_sample_count
                                      , all_items=all_index
                                      , user_all_items=user_all_items
                                      , shuffle=False)
          #Training and test for every epoch
          for epoch in range(FLAGS.num_epochs):
              cost_list = []
              for train_input in trainIterator:
                user, next_target, user_seq, sl, neg_seq = train_input

                # Convert lists to NumPy arrays before checking shape
                user_seq_array = np.array(user_seq)
                neg_seq_array = np.array(neg_seq)
                user_array = np.array(user)
                next_target_array = np.array(next_target)
                # Print shapes of relevant tensors
                # print("Shape of user_seq:", user_seq_array.shape)
                # print("Shape of neg_seq:", neg_seq_array.shape)
                # print("Shape of hist_seq:", user_seq_array.shape)  #tis the issue
                feed_dict = {model.u_p: user, model.next_p: next_target, model.sl: sl,
                            model.hist_seq: user_seq, model.neg_p: neg_seq,
                            model.keep_prob:FLAGS.keep_prob,model.is_Training:True}

                _, step, cost = sess.run([train_op, global_step, total_loss], feed_dict)
                cost_list.append(np.mean(cost))
              mean_cost = np.mean(cost_list)
              saver.save(sess, FLAGS.save_path)

              pred_list = []
              next_list = []
              # test and cal hr50 and mrr
              for test_input in testIterator:
                  user, next_target, user_seq, sl, neg_seq = test_input
                  feed_dict = {model.u_p: user, model.next_p: next_target, model.sl: sl,
                              model.hist_seq: user_seq,model.keep_prob:1.0
                              ,model.is_Training:False}
                  pred_indexs = sess.run(topk_index, feed_dict)
                  pred_list += pred_indexs.tolist()
                  #only predict one next item
                  single_target = [item[0] for item in next_target]
                  next_list += single_target
              hr50 = Metric_HR(next_list,pred_list,50)
              mrr = Metric_MRR(next_list,pred_list)
              print(" epoch {},  mean_loss{:g}, test HR@50: {:g}, test MRR: {:g}"
                    .format(epoch + 1, mean_cost,hr50,mrr))



if __name__ == '__main__':
    main([])

 make datasets
Train and test datasets saved in 'processed_data'
       user                        seq            target
0         0    [0, 289, 491, 380, 751]    [466, 522, 10]
1         0  [289, 491, 380, 751, 466]    [522, 10, 672]
2         0  [491, 380, 751, 466, 522]   [10, 672, 1045]
3         0   [380, 751, 466, 522, 10]  [672, 1045, 649]
4         0   [751, 466, 522, 10, 672]  [1045, 649, 377]
...     ...                        ...               ...
92451   942   [209, 10, 873, 935, 614]    [355, 158, 12]
92452   942   [10, 873, 935, 614, 355]    [158, 12, 141]
92453   942  [873, 935, 614, 355, 158]    [12, 141, 452]
92454   942   [935, 614, 355, 158, 12]   [141, 452, 672]
92455   942   [614, 355, 158, 12, 141]    [452, 672, 68]

[92456 rows x 3 columns]
     user                           seq            target
0       0    [834, 438, 632, 656, 1006]   [947, 363, 521]
1       1       [452, 899, 25, 246, 48]    [530, 145, 31]
2       2     [758, 437, 458, 476, 368]    [769, 14

W0000 00:00:1732095993.559160 1959595 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
W0000 00:00:1732096001.458769 1959595 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 26843

 epoch 1,  mean_loss0.13587, test HR@50: 0.424178, test MRR: 0.0508215
 epoch 2,  mean_loss0.0641207, test HR@50: 0.460233, test MRR: 0.0531281
 epoch 3,  mean_loss0.0547577, test HR@50: 0.495228, test MRR: 0.0573095
 epoch 4,  mean_loss0.0518686, test HR@50: 0.504772, test MRR: 0.0623951
 epoch 5,  mean_loss0.0509257, test HR@50: 0.515376, test MRR: 0.0631909
 epoch 6,  mean_loss0.0499196, test HR@50: 0.520679, test MRR: 0.0558318
 epoch 7,  mean_loss0.0491972, test HR@50: 0.5228, test MRR: 0.0592131
 epoch 8,  mean_loss0.0488362, test HR@50: 0.531283, test MRR: 0.0646016
 epoch 9,  mean_loss0.0484905, test HR@50: 0.532344, test MRR: 0.0645264
 epoch 10,  mean_loss0.0481041, test HR@50: 0.528102, test MRR: 0.0627065
 epoch 11,  mean_loss0.0476032, test HR@50: 0.535525, test MRR: 0.0681188
 epoch 12,  mean_loss0.0478272, test HR@50: 0.52492, test MRR: 0.0649222
 epoch 13,  mean_loss0.0474808, test HR@50: 0.535525, test MRR: 0.0664004
 epoch 14,  mean_loss0.0474026, test HR@50: 0.525981

## References

1. S. Zhang, W. Chen, and H. Lee, "Next item recommendation with self-attention," in Proceedings of the 43rd International ACM SIGIR Conference on Research and Development in Information Retrieval, 2020, pp. 1227–1236, doi: 10.1145/3397271.3401075.
2. S. Ge, "AttRec: A Recommender System with Self-Attention Mechanism," GitHub repository, 2020. [Online]. Available: https://github.com/slientGe/AttRec. [Accessed: Nov. 19, 2024].

<i>Copyright (c) Recommenders contributors.</i> [GitHub](https://github.com/recommenders-team/recommenders)

<i>Licensed under the MIT License.</i>

<i>Implementation repo: [GitHub](https://github.com/LeeIsBadK/AttRec-implement-project) </i>